# Multilevel Multigrid Example

In [1]:
# dependencies
using LFAToolkit
using LinearAlgebra
using Pkg
Pkg.activate("./")
Pkg.instantiate()
using Plots

 Activating environment at `~/Dev/LFAToolkit.jl/examples/jupyter/Project.toml`


In [2]:
# setup
finep = 9
midp5 = 5
midp3 = 3
coarsep = 2
dimension = 2

mesh = []
if dimension == 1
   mesh = Mesh1D(1.0)
elseif dimension == 2
   mesh = Mesh2D(1.0, 1.0)
end

mtofbasis5 = TensorH1LagrangeBasis(midp5, finep, dimension, lagrangequadrature=true)
mtofbasis3 = TensorH1LagrangeBasis(midp3, midp5, dimension, lagrangequadrature=true)
ctombasis = TensorH1LagrangeBasis(coarsep, midp3, dimension, lagrangequadrature=true)

# diffusion operators
finediffusion = GalleryOperator("diffusion", finep, finep, mesh)
middiffusion5 = GalleryOperator("diffusion", midp5, finep, mesh)
middiffusion3 = GalleryOperator("diffusion", midp3, finep, mesh)
coarsediffusion = GalleryOperator("diffusion", coarsep, finep, mesh)

# Jacobi smoother
finejacobi = Jacobi(finediffusion)
midjacobi5 = Jacobi(middiffusion5)
midjacobi3 = Jacobi(middiffusion3)

# p-multigrid preconditioner
midmultigrid3 = PMultigrid(middiffusion3, coarsediffusion, midjacobi3, [ctombasis])
midmultigrid5 = PMultigrid(middiffusion5, midmultigrid3, midjacobi5, [mtofbasis3])
multigrid = PMultigrid(finediffusion, midmultigrid5, finejacobi, [mtofbasis5])

p-multigrid preconditioner

In [18]:
# full operator symbols
numberruns = 100
numberruns_w = 1
maxeigenvalue = 0
θ_min = -π/2
θ_max = 3π/2
ω_max = 1.0

# compute and plot smoothing factor
# -- 1D --
if dimension == 1
    # setup
    smoothingfactor = zeros(numberruns)
    ω_min = [0.0, 1.0]
    v = [2, 2]

    # compute
    for i in 1:numberruns
        ω = [0.5+(ω_max-0.5)*i/numberruns]
        for j in 1:numberruns
            θ = [θ_min + (θ_max - θ_min)*j/numberruns]
            if abs(θ[1] % 2π) >  π/512
                A = computesymbols(multigrid, ω, v, θ)
                eigenvalues = [abs(val) for val in eigvals(A)]
                smoothingfactor[i] = max(smoothingfactor[i], eigenvalues...)
            end
        end
        if smoothingfactor[i] < ω_min[2]
            ω_min = [ω[1], smoothingfactor[i]]
        end
    end

    # plot
    println("Min ω: ", ω_min)
    xrange = 0:ω_max/(numberruns-1):ω_max
    plot(xrange,
        smoothingfactor,
        xlabel="ω",
        xtickfont=font(12, "Courier"),
        ylabel="spectral radius",
        ytickfont=font(12, "Courier"),
        linewidth=3,
        legend=:none,
        title="Multigrid Convergence Factor"
    )
    ylims!(0.0, max(smoothingfactor...))
    savefig("multi_grid_converge_9_to_2_2smooth")
# -- 2D --
elseif dimension == 2
    # setup
    smoothingfactor = zeros(numberruns_w)
    ω_min = [0.0, 1.0]
    v = [1, 1]

    # compute
    for i in 1:numberruns_w
        ω = [0.820]
        for j in 1:numberruns, k in 1:numberruns
            θ = [
                θ_min + (θ_max - θ_min)*j/numberruns,
                θ_min + (θ_max - θ_min)*k/numberruns
            ]
            if θ[1] > π/2 || θ[2] > π/2
                A = computesymbols(multigrid, ω, v, θ)
                eigenvalues = [abs(val) for val in eigvals(A)]
                smoothingfactor[i] = max(smoothingfactor[i], eigenvalues...)
                if smoothingfactor[i] < ω_min[2]
                    ω_min = [ω[1], smoothingfactor[i]]
                end
            end
        end
    end

    # plot
    println("Min ω: ", ω_min)
    xrange = 0:ω_max/(numberruns_w-1):ω_max
    plot(xrange,
        smoothingfactor,
        xlabel="ω",
        xtickfont=font(12, "Courier"),
        ylabel="spectral radius",
        ytickfont=font(12, "Courier"),
        linewidth=3,
        legend=:none,
        title="Multigrid Convergence Factor"
    )
    ylims!(0.0, max(smoothingfactor...))
    savefig("multi_grid_converge_9_to_2_2smooth_2d")
end

Min ω: [0.82, 0.6073370004492418]


In [4]:
ω_min

2-element Array{Float64,1}:
 0.615
 0.6945309334026675

In [ ]:
Min ω: [0.615, 0.6945309334026675]
Min ω: [0.701, 0.6562906364159404]
Min ω: [0.751, 0.6346582079215057]
Min ω: [0.775, 0.6244512055079867]
Min ω: [0.801, 0.6135442110017193]
Min ω: [0.805, 0.6118827366631115]
Min ω: [0.815, 0.6077616114588631]
Min ω: [0.817, 0.60695559063608]
Min ω: [0.818, 0.6065690462596208]

Min ω: [0.819, 0.6062548678596762]

Min ω: [0.820, 0.6073370004492418]
Min ω: [0.821, 0.6104950274628109]
Min ω: [0.825, 0.6239916426990797]
Min ω: [0.851, 0.7161106898022598]
Min ω: [0.900, 0.9071364009489794]